# 🪖 Helmet Detection for Bike Riders — Week 2

### 🎯 Objective (Week 2)
- Prepare a **custom 2-class dataset**: `helmet` and `no-helmet`
- **Fine-tune YOLOv8** on this dataset
- Evaluate with **precision/recall, mAP** and confusion matrix
- Run **batch inference** on images & videos
- Optional: **real-time webcam demo**
- Export model to **ONNX/TFLite** for deployment

> This notebook assumes you either:
> 1) Have a local dataset in YOLO format (recommended), or
> 2) Will download one via Roboflow (optional).

In [1]:
# ------------------------------------------------------------
# 1) Install dependencies
# ------------------------------------------------------------
!pip install -q ultralytics opencv-python matplotlib ipywidgets

import sys, ultralytics, cv2, matplotlib
print("Python:", sys.version.split("\n")[0])
print("Ultralytics:", ultralytics.__version__)
print("OpenCV:", cv2.__version__)
print("Matplotlib:", matplotlib.__version__)

Python: 3.13.7 (tags/v3.13.7:bcee1c3, Aug 14 2025, 14:15:11) [MSC v.1944 64 bit (AMD64)]
Ultralytics: 8.3.223
OpenCV: 4.12.0
Matplotlib: 3.10.7


In [2]:
# ------------------------------------------------------------
# 2) Imports & paths
# ------------------------------------------------------------
from ultralytics import YOLO
import os, yaml
from pathlib import Path
from IPython.display import display
import matplotlib.pyplot as plt

ROOT = Path.cwd()
DATA_DIR = ROOT / "data_week2"
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Expected structure if you have a local dataset in YOLO format:
# data_week2/
#   ├── images/
#   │   ├── train/*.jpg|png
#   │   └── val/*.jpg|png
#   ├── labels/
#   │   ├── train/*.txt
#   │   └── val/*.txt
#   └── data.yaml  (we will create if missing)

print("Project root:", ROOT)
print("Data directory:", DATA_DIR)

Project root: C:\Users\varun kumar
Data directory: C:\Users\varun kumar\data_week2


## Option A — Use **local dataset** (recommended)
Place your dataset in `data_week2/images` and `data_week2/labels` (YOLO format). Then run the cell below to create `data.yaml` automatically.

In [3]:
# ------------------------------------------------------------
# 3A) Create data.yaml for LOCAL dataset (2 classes)
# ------------------------------------------------------------
images_train = DATA_DIR / "images" / "train"
images_val   = DATA_DIR / "images" / "val"
labels_train = DATA_DIR / "labels" / "train"
labels_val   = DATA_DIR / "labels" / "val"

yaml_path = DATA_DIR / "data.yaml"
data_cfg = {
    'path': str(DATA_DIR.resolve()),
    'train': 'images/train',
    'val': 'images/val',
    'names': ['helmet', 'no-helmet']
}

yaml_path.write_text(yaml.dump(data_cfg))
print("✅ Wrote:", yaml_path)
print(yaml_path.read_text())

for p in [images_train, images_val, labels_train, labels_val]:
    p.mkdir(parents=True, exist_ok=True)
    print("Ensured:", p)

✅ Wrote: C:\Users\varun kumar\data_week2\data.yaml
names:
- helmet
- no-helmet
path: C:\Users\varun kumar\data_week2
train: images/train
val: images/val

Ensured: C:\Users\varun kumar\data_week2\images\train
Ensured: C:\Users\varun kumar\data_week2\images\val
Ensured: C:\Users\varun kumar\data_week2\labels\train
Ensured: C:\Users\varun kumar\data_week2\labels\val


## Option B — Download a dataset from Roboflow (optional)
1. Create a free Roboflow account and get an API key
2. Replace `YOUR_API_KEY` and `WORKSPACE/PROJECT:VERSION`
3. Run the cell to auto-download in YOLOv8 format

_Skip if you already have a local dataset._

In [4]:
# ------------------------------------------------------------
# 3B) (Optional) Roboflow download to data_week2/
# ------------------------------------------------------------
use_roboflow = False  # set True after adding your API key & project
if use_roboflow:
    !pip install -q roboflow
    from roboflow import Roboflow
    rf = Roboflow(api_key="YOUR_API_KEY")
    project = rf.workspace("WORKSPACE").project("PROJECT")
    dataset = project.version(VERSION_NUMBER).download("yolov8", location=str(DATA_DIR))
    print("Downloaded to:", dataset.location)
    # If roboflow creates its own folder, copy/move data.yaml up to data_week2/data.yaml if needed
else:
    print("Skipping Roboflow download. Using local dataset.")

Skipping Roboflow download. Using local dataset.


## Quick dataset sanity check
- Counts images and labels
- Warns about potential missing labels
- Shows a couple of sample images if present

In [5]:
import glob
import random

def count_files(folder, exts=(".jpg", ".jpeg", ".png")):
    return sum(1 for p in Path(folder).rglob("*") if p.suffix.lower() in exts)

n_train_img = count_files(images_train)
n_val_img   = count_files(images_val)
n_train_lab = len(list(labels_train.rglob("*.txt")))
n_val_lab   = len(list(labels_val.rglob("*.txt")))

print(f"Train images: {n_train_img} | labels: {n_train_lab}")
print(f"Val images  : {n_val_img} | labels: {n_val_lab}")
if n_train_img == 0 or n_val_img == 0:
    print("⚠️ Add images to data_week2/images/train and images/val before training.")

# Show a couple of samples if available
sample_paths = list(Path(images_train).rglob("*.jpg")) + list(Path(images_train).rglob("*.png"))
random.shuffle(sample_paths)
for p in sample_paths[:2]:
    import cv2, matplotlib.pyplot as plt
    img = cv2.imread(str(p))
    if img is None: continue
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(6,4))
    plt.imshow(img)
    plt.title(p.name)
    plt.axis('off')
    plt.show()

Train images: 0 | labels: 0
Val images  : 0 | labels: 0
⚠️ Add images to data_week2/images/train and images/val before training.


## Train — Fine-tune YOLOv8n on 2 classes
You can tweak `epochs`, `imgsz`, `batch`, and `lr0`. Early stopping via `patience`.

In [ ]:
# ------------------------------------------------------------
# 4️⃣ Training (with your real dataset)
# ------------------------------------------------------------
from ultralytics import YOLO

# Load the pretrained YOLOv8n model
model = YOLO("yolov8n.pt")

# Train using your downloaded dataset
results = model.train(
    data="C:/Users/varun kumar/Downloads/Motorcycle Helmet Detection.v2i.yolov8/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    patience=10,
    lr0=0.01,
    weight_decay=0.0005,
    project="runs_week2",
    name="helmet_yolov8n",
    verbose=True
)

print("\n✅ Training complete. Best weights saved to:", model.trainer.best)


New https://pypi.org/project/ultralytics/8.3.226 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.223  Python-3.13.7 torch-2.9.0+cpu CPU (13th Gen Intel Core i7-1360P)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/varun kumar/Downloads/Motorcycle Helmet Detection.v2i.yolov8/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_sc

## Validate — Metrics & confusion matrix

In [ ]:
# ------------------------------------------------------------
# 5) Validation on the best weights
# ------------------------------------------------------------
best_weights = Path(model.trainer.best)
best_model = YOLO(str(best_weights))
metrics = best_model.val(data=str(yaml_path), imgsz=640, batch=16)
print(metrics)  # shows mAP50-95, precision, recall, etc.

# Display confusion matrix if generated
conf_mat = best_model.ckpt_path and (best_weights.parent / "confusion_matrix.png")
cm_candidates = list(best_weights.parent.rglob("confusion_matrix.png"))
if cm_candidates:
    from IPython.display import Image
    print("Confusion matrix:")
    display(Image(filename=str(cm_candidates[0])))
else:
    print("(Confusion matrix image not found in run folder.)")

## Inference — Batch on a folder of images

In [ ]:
# ------------------------------------------------------------
# 6) Batch inference on any images folder
# ------------------------------------------------------------
INFER_DIR = ROOT / "demo_images"
INFER_DIR.mkdir(exist_ok=True)
print("Put some .jpg/.png images into:", INFER_DIR)

pred = best_model.predict(source=str(INFER_DIR), save=True, imgsz=640, conf=0.25)
print("Saved predictions to:", pred[0].save_dir if pred else "<none>")

## Video inference — on a file

In [ ]:
# ------------------------------------------------------------
# 7) Video inference on a video file
# ------------------------------------------------------------
VIDEO_PATH = str((ROOT / "sample.mp4").resolve())  # change to your path
if Path(VIDEO_PATH).exists():
    _ = best_model.predict(source=VIDEO_PATH, save=True, imgsz=640, conf=0.25)
else:
    print("No video found at:", VIDEO_PATH)

## Real-time webcam demo (optional)

In [ ]:
# ------------------------------------------------------------
# 8) Webcam demo (press 'q' to quit)
# ------------------------------------------------------------
import cv2

def show_webcam(model, cam_index=0, conf=0.25):
    cap = cv2.VideoCapture(cam_index)
    if not cap.isOpened():
        print("⚠️ Could not open webcam.")
        return
    try:
        while True:
            ok, frame = cap.read()
            if not ok:
                break
            results = model.predict(frame, imgsz=640, conf=conf, verbose=False)
            annotated = results[0].plot()
            cv2.imshow("Helmet Detection (q to quit)", annotated)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

# Uncomment to run locally
# show_webcam(best_model, cam_index=0, conf=0.35)

## Export model — ONNX/TFLite

In [ ]:
# ------------------------------------------------------------
# 9) Export formats
# ------------------------------------------------------------
onnx_path = best_model.export(format="onnx")
print("Exported ONNX:", onnx_path)

try:
    tflite_path = best_model.export(format="tflite")
    print("Exported TFLite:", tflite_path)
except Exception as e:
    print("TFLite export not available in this environment:", e)

---
### 🧠 GitHub README (Week 2)
```markdown
# Helmet Detection — Week 2

## Goals
- Fine-tune YOLOv8 for 2 classes: `helmet`, `no-helmet`
- Evaluate (mAP, P/R) and confusion matrix
- Image/Video/Real-time inference
- Export to ONNX/TFLite

## Dataset (YOLO format)
data_week2/
  ├── images/{train,val}
  ├── labels/{train,val}
  └── data.yaml

## Train
```python
from ultralytics import YOLO
model = YOLO('yolov8n.pt')
model.train(data='data_week2/data.yaml', epochs=50, imgsz=640)
```

## Inference
```python
best = YOLO('runs_week2/helmet_yolov8n/weights/best.pt')
best.predict('demo_images', save=True)
```
```

In [1]:
import os
print(os.getcwd())


C:\Users\varun kumar
